In [ ]:
import sys

sys.path.append("..")
from common_utils import set_data_home

set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from sales_data import Sales_Dataset
import torch
import random

device = "cuda" if torch.cuda.is_available() else "cpu"
SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")

torch.manual_seed(42)
random.seed(42)

saved_iter = 500
I, H, B = 528, 528, 4
TRANSFORMER_LAYER = 4
LSTM_LAYER = 2
HEAD = 12
SEQ_LEN = 116
INFER_DAYS = 16

In [ ]:
from predictor import Predictor

model = Predictor(I, H, LSTM_LAYER, TRANSFORMER_LAYER, HEAD, SEQ_LEN).cuda()

In [ ]:
sd = Sales_Dataset(SALE_HOME, seq_len=SEQ_LEN, is_train=False)

In [ ]:
print(sd[1][0].shape, sd[1][1].shape, sd[1][2].shape, sd[1][3])

### Perform Inference

In [ ]:
import pandas as pd
from datetime import timedelta

model.load_state_dict(torch.load(f"sales_model_{saved_iter}_{B}.pth"))

sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=0)
sales["sales"] = 0.0
# base_sales = sd.base_sales.set_index(["store_nbr", "date"])

for X1, X2, base_sales, store_id, di in sd:
    X1 = X1.cuda().unsqueeze(0)
    X2 = X2.cuda().unsqueeze(0)
    base_sales = base_sales[[-1]].cuda().unsqueeze(0)
    di = ([di[0]], [di[1]])
    yhat1, yhat2 = model(X1, X2, di)

    # infer and update input for each store
    yhat_sales = Sales_Dataset.batched_ret_2_sales(
        base_sales, yhat1[:, [-1], :], yhat2[:, [-1], :]
    )
    yhat_sales = yhat_sales.squeeze(0, 1)
    for i in range(INFER_DAYS):
        # compute actual sales
        ts = (sd.train_max_date + timedelta(days=i + 1)).strftime("%Y-%m-%d")

        # write to each family in the answer dataframe
        for j, f in enumerate(sd.families):
            sales.loc[
                (sales.date == ts)
                & (sales.store_nbr == store_id)
                & (sales.family == f),
                "sales",
            ] = (
                yhat_sales[i * len(sd.families) + j].cpu().item()
            )

### output the answer

In [ ]:
sales

In [ ]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values(
    "id"
).sort_index().to_csv("answer.csv", index=True)